# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Action Points

### 1. No need to make 10 transactions a day. Just one is enough
### 2. Close diff (Movement of close value over a day) looks promising to find trend or ranging market

data                = get_date_list(data)
data                = read_data(data)

In [2]:
%%time
data = run_engine(data)

 99%|█████████████████████████████████▊| 73871/74266 [00:18<00:00, 3704.80it/s]

pl   -0.06616
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
67428 2021-11-30 04:00:00         NaN  sema_close -0.00373
67732 2021-12-01 05:20:00         NaN  sema_close  0.00179
68514 2021-12-05 22:30:00         NaN  sema_close -0.00109
69179 2021-12-08 05:55:00         NaN  sema_close -0.00119
69573 2021-12-09 14:45:00         NaN  sema_close  0.00065
69948 2021-12-12 22:05:00         NaN  sema_close -0.00297
70427 2021-12-14 14:00:00         NaN  sema_close -0.00249
70847 2021-12-16 01:00:00         NaN  sema_close -0.00185
71332 2021-12-17 17:25:00         NaN  sema_close -0.00135
71615 2021-12-20 17:00:00         NaN  sema_close -0.00438
71986 2021-12-21 23:55:00         NaN  sema_close  0.00015
72562 2021-12-23 23:55:00         NaN  sema_close  0.00167
73103 2021-12-27 21:05:00         NaN  sema_close -0.00210
73651 2021-12-29 18:45:00         NaN  sema_close -0.00480
73863 2021-12-30 12:25:00         NaN  sema_clo

100%|██████████████████████████████████████| 257/257 [00:00<00:00, 1111.35it/s]

Wall time: 1min 3s


In [3]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1    -0.01931
2     0.01145
3     0.00564
4    -0.02115
5     0.00028
6    -0.00415
7    -0.01675
8     0.01417
9    -0.00901
10   -0.00781
11    0.00072
12   -0.02024
Name: pl, dtype: float64
---------------------
max : 8 : 0.01417
min : 4 : -0.021150000000000002
---------------------
monthly_avg : -0.0055


In [4]:
data['lema_gap_pl_analysis_month'] = 8

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
2,0.000,2.0,0.0,2.0,0.00453,0.00000,0.00453
1,0.001,3.0,0.0,3.0,0.01395,0.00000,0.01395
0,0.002,0.0,2.0,-2.0,0.00000,-0.00431,-0.00431


In [5]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
4,0.000,1.0,1,0.0,0.00272,-0.00199,0.00073
2,0.001,0.0,1,-1.0,0.00000,-0.00534,-0.00534
1,0.002,1.0,1,0.0,0.01527,-0.00700,0.00827
0,0.003,1.0,2,-1.0,0.00874,-0.00250,0.00624
3,0.004,0.0,1,-1.0,0.00000,-0.00284,-0.00284


In [6]:
sys.exit()

SystemExit: 

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [7]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
month = 4

# Monthly Chart -------------------
data['plot_start']  = f"{data['input_year']}-{month}-1"
data['plot_stop']   = f"{data['input_year']}-{month+1}-1"
# Monthly Chart -------------------

# December Chart ----------------------------
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# December Chart ----------------------------

# Year Chart ----------------------------
# data['plot_start']  = f"{data['input_year']}-1-1"
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# Year Chart ----------------------------

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
data['things_to_plot'] = ['candles','sema', 'slema','lema', 'lema_angle', 'lema_max_min', 'close_diff']

plot_graph(data)

generate_result_report(data)

In [ ]:
data['df_ohlc']['close_diff'].describe()